In [7]:
# Spinglass(tau = 1e3)

import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
tau = torch.tensor(100, device=device)
evaporation_rate = torch.exp(-1 / tau)
start_seed = 43
start_alpha = 0.0
#seed = start_seed
iter = 1e7
energy_series = []
m_series = []

spins, h, J, st, st1, alpha, alpha_inc = simulation.initialize_spinglass(n, start_seed, iter, start_alpha, device)

for i in range(int(iter)):
    spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st = st * evaporation_rate + torch.exp(-simulation.calculate_energy(spins, h, J))
    st1 = st1 * evaporation_rate + torch.exp(-simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
    z = st1 / st
    alpha = min(alpha + alpha_inc, 0.999)
    m = 2 * alpha * (z - 0.5)
    if i % int(iter / 1e6) == 0:
        energy_series.append(simulation.calculate_energy(spins, h, J).cpu().item())
        m_series.append(m.tolist())

energy_series.insert(0, 'energy')
energy_file_name = f'tau{tau:.0e}_spinglass_annealing_energy'
m_file_name = f'tau{tau:.0e}_spinglass_annealing_m'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)

In [ ]:
# Spinglass(tau = 1e3) modified

import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
tau = torch.tensor(1000, device=device)
evaporation_rate = torch.exp(-1 / tau)
start_seed = 42
start_alpha = 0.5
#seed = start_seed
iter = 1e6
energy_series = []
m_series = []

spins, h, J, st, st1, alpha_inc, alpha = simulation.initialize_spinglass_modified(n, start_seed, iter, start_alpha, device)

for i in range(int(iter)):
    spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st = st * evaporation_rate + torch.exp(-simulation.calculate_energy(spins, h, J))
    st1 = st1 * evaporation_rate + torch.exp(-simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
    z = st1 / st
    alpha = min(alpha + alpha_inc, 0.999)
    m = 2 * alpha * (z - 0.5)
    energy_series.append(simulation.calculate_energy(spins, h, J).cpu().item())
    m_series.append(m.tolist())

energy_series.insert(0, 'energy')
energy_file_name = f'tau{tau:.0e}_start{start_alpha}_spinglass_energy'
m_file_name = f'tau{tau:.0e}_start{start_alpha}_spinglass_m'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)

In [ ]:
#Homogeneous fully connected ising model(tau=1e3)

import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
tau = torch.tensor(1000, device=device)
evaporation_rate = torch.exp(-1 / tau)
start_seed = 42
#seed = start_seed
iter = 1e6
energy_series = []
m_series = []

spins, h, J, st, st1, alpha, alpha_inc = simulation.initialize_homogeneous(n, start_seed, iter, device)

for i in range(int(iter)):
    spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st = st * evaporation_rate + torch.exp(-simulation.calculate_energy(spins, h, J))
    st1 = st1 * evaporation_rate + torch.exp(-simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
    z = (st1 / st)
    alpha = min(alpha + alpha_inc, 0.999)
    m = 2 * alpha * (z - 0.5)
    energy_series.append(simulation.calculate_energy(spins, h, J).cpu().item())
    m_series.append(m.tolist())

energy_series.insert(0, 'energy')
energy_file_name = f'tau{tau:.0e}_homo_energy'
m_file_name = f'tau{tau:.0e}_homo_m'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)